In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.metrics import MeanSquaredError

In [26]:
data = pd.read_csv('./input/copper_returns_5d_final.csv')
data.describe()

,LMCADS03,LMCADY,DXY,SPX,BCOM,MXWD,XAU,XAG,LMCADY_acu_5d,LMCADY_std_5d
count,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000
mean,-0.000185,-0.000178,0.000007,-0.000252,0.000072,-0.000205,-0.000274,-0.000106,-0.000867,0.014218
std,0.016059,0.016464,0.004793,0.011794,0.010250,0.009880,0.010722,0.019587,0.034782,0.008721
min,-0.112019,-0.110645,-0.024921,-0.103782,-0.054910,-0.085172,-0.097378,-0.123485,-0.185825,0.000805
25%,-0.008649,-0.008880,-0.002635,-0.005469,-0.005554,-0.004878,-0.005978,-0.009770,-0.021102,0.008616
50%,0.000000,0.000000,0.000000,-0.000390,0.000000,-0.000643,-0.000497,-0.000771,-0.001844,0.012148
75%,0.007562,0.007760,0.002658,0.003949,0.005375,0.003807,0.004908,0.008074,0.017065,0.017520
max,0.109603,0.109134,0.027541,0.136158,0.066117,0.105134,0.099792,0.226116,0.259832,0.091981


In [27]:
features = data.drop(columns=['Date','LMCADY_std_5d','LMCADY_acu_5d'])
target = data['LMCADY_acu_5d']

# desviacion estandar del target
std_target = target.std()
std_target

0.034782023369004565

In [28]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

In [29]:
time_steps = 1
X, y = create_dataset(features, target, time_steps)

In [30]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [31]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

In [32]:
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

In [33]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[rmse])

In [34]:
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1, verbose=1)


Epoch 1/30


125/125 [==============================] - 1s 2ms/step - loss: 8.9338e-04 - rmse: 0.0294 - val_loss: 0.0025 - val_rmse: 0.0440
Epoch 2/30
125/125 [==============================] - 0s 2ms/step - loss: 8.2369e-04 - rmse: 0.0282 - val_loss: 0.0024 - val_rmse: 0.0438
Epoch 3/30
125/125 [==============================] - 0s 1ms/step - loss: 8.1507e-04 - rmse: 0.0280 - val_loss: 0.0025 - val_rmse: 0.0443
Epoch 4/30
125/125 [==============================] - 0s 2ms/step - loss: 8.0635e-04 - rmse: 0.0279 - val_loss: 0.0025 - val_rmse: 0.0443
Epoch 5/30
125/125 [==============================] - 0s 2ms/step - loss: 8.0967e-04 - rmse: 0.0279 - val_loss: 0.0024 - val_rmse: 0.0435
Epoch 6/30
125/125 [==============================] - 0s 2ms/step - loss: 8.0401e-04 - rmse: 0.0278 - val_loss: 0.0025 - val_rmse: 0.0438
Epoch 7/30
125/125 [==============================] - 0s 2ms/step - loss: 8.0560e-04 - rmse: 0.0280 - val_loss: 0.0025 - val_rmse: 0.0442
Epoch 8/30
125/125 [=========================

In [35]:
loss, rmse = model.evaluate(X_test, y_test)

rmse

35/35 [==============================] - 0s 712us/step - loss: 0.0012 - rmse: 0.0320


0.032014623284339905